# Numerične metode v tehniki
## Optimizacija - 1.del

- **avtor**: A.S. Grm
- **date**: 18/11/2023
- **primer**: 01/1

<hr>

## Metoda najmanjših kvadratov - Linearni problem

Imamo meriritve hitrosti in sile pri prostem padu telesa določene oblike. Če velja linearen zakon upora

$$ F_d = \alpha v,$$

ali pa kvadraten zakon upora

$$ F_d = \alpha v^2 $$

bi radi iz meritev dobili vrednost konstante $\alpha$?

Kot metodo reševanja uporabi **metodo najmanjših kvadratov**, kjer za modelsko funkcijo uporabiš polinom

$$ p_n(x) = a_0 + a_1 x + a_2 x^2 + ... + a_n x^n $$

<hr>

Reševanje **normalnega sistema**

$$A^T A x = A^T b$$

z uporabo razcepa Choleskega

- izračunaj $B = A^T A$ in $c = A^T b$
- izračunaj razcep Choleskega $B = V V^T$
- reši spodnje trikotni sistem $V y = c$
- reši zgornje trikotni sistem $V^T x = y$
  
<hr>

### Napaka modelske funkcije - $R^2$ (*Coefficient of determination*)

<br>
V regresijski analizi bi radi preverili, kako dober je naš model za prilagajanje podatkom, kjer označimo merske točke $P_i(x_i,y_i)$. Mera kvalitete je v tako imenovani **koeficient determinacije**, ki ga označimo z $R^2$ ($R$ squared). Določimo ga po naslednjem postopku

<br><br>
- najprej izračunamo povprečje
$$ \bar{y} = \frac{1}{n} \sum_{i=1}^n y_i $$
- nato poračunamo residual
$$ S_{\text{res}} = \sum_{i=1}^n \big(y_i - f(x_i) \big)^2 $$
- celotno razliko od povprečja
$$ S_{\text{tot}} = \sum_{i=1}^n \big(y_i - \bar{y} \big)^2. $$

Napaka modelske funkcije je nato določena z

$$ R^2 = 1 - \frac{S_{\text{res}}}{S_{\text{tot}}} $$

<hr>

In [ ]:
import math as mat
import numpy as np
import matplotlib.pyplot as mpl

# MatPlotLib set fonts
mpl.rcParams['font.family'] = 'serif'
mpl.rcParams['font.serif'] = ['DejaVu Serif']

# MatPlotLib set LaTeX use
mpl.rcParams['text.usetex'] = True
mpl.rcParams['text.latex.preamble'] = r'\usepackage{siunitx}'

In [ ]:
# Data

v = [0,1.38,1.99,2.2,2.51,3.06,3.77,4.09,5.51,6.21,7.22,7.88,8.53,9.79,10.31,10.93,11.21,11.37]
Fd = [0.0,0.1,0.3,0.3,0.4,0.5,0.7,0.8,1.3,1.9,2.3,2.6,2.9,3.8,4.1,4.4,4.7,4.9]

In [ ]:
fig, ax = mpl.subplots()
#fig.suptitle(r'Meritev upora pri prostem padu') # Figure title

ax.plot(v,Fd, '.g')
ax.set_xlabel(r'$v$ [m/s]')
ax.set_ylabel(r'$F_d$ [N]')
ax.grid()
ax.set_aspect('equal')

fig.tight_layout()

In [ ]:
# Reši normalni sistem, kjer je samo en modelski koeficient

def solve_ns_one(ym,bm,k):

    yv = np.array(ym)**k
    bv = np.array(bm)
    
    A = np.transpose(np.array(yv))
    B = np.matmul(np.transpose(A),A)
    c = np.matmul(np.transpose(A),bv)

    x = c/B
    
    return x

In [ ]:
# Reši normalni sistem, kjer imamo več modelskih koeficientov

def solve_ns_high(pn,ym,bm):

    mm = pn.size
    nn = yv.size

    poly_data = []
    for i in range(mm):
        yi = np.array(ym)**i
        if pn[i] == 1:
            poly_data.append(yi)
        else:
            poly_data.append(np.zeros(nn))

    pd = np.array(poly_data)
    
    A = np.transpose(pd)
    B = np.matmul(np.transpose(A),A)
    V = np.linalg.cholesky(B)
    c = np.matmul(np.transpose(A),bv)

    y = np.linalg.solve(V,c)
    x = np.linalg.solve(np.transpose(V),y)
    
    return x

In [ ]:
# Reši problem najmanjših kvadratov, kjer imamo polinomsko modelsko funkcijo
# pn(x) = a_0 + a_1 x + a_2 x^2 + ... + a_n x^n

def solve_lpls(pn,y,b):

    pv = np.array(pn)
    ones = np.count_nonzero(pv == 1)

    yv = np.array(y)
    bv = np.array(b)

    if ones == 1:
        k = np.where(pv == 1)[0][0]
        x = solve_ns_one(yv,bv,k)
    else:
        x = solve_ns_high(pv,yv,bv)

    return x

In [ ]:
# Vrne točke na modelski krivulji pn(x)

def find_model_function(pn,ai,x):

    pnv = np.array(pn)
    ones = np.count_nonzero(pnv == 1)

    if ones == 1:
        k = np.where(pnv == 1)[0][0]
        y = ai * x**k
    else:    
        y = 0
        n = 0
        for k in pn:
            if k == 1:
                y += ai[n] * x**n
            n += 1

    return y

In [ ]:
# Napaka modela R2

def R2(xi, yi, pn, a):

    xv = np.array(xi)
    yv = np.array(yi)
    ym = np.mean(yv)
    fi = find_model_function(pn,a,xi)

    Sr = np.sum(np.square(yv - fi))
    St = np.sum(np.square(yv - ym))

    R2 = 1 - Sr/St

    return R2

In [ ]:
# Glavni program

pn1 = [0,1] # pn(x) = 0 + a_1*x
x1 = solve_lpls(pn1,v,Fd) # reši linearni upor

pn2 = [0,0,1] # pn(x) = 0 + 0*x + a_2*x^2
x2 = solve_lpls([0,0,1],v,Fd) # reši kvadratičen upor

# izračuna točke krivulje upora
vi = np.linspace(v[0],v[-1],100)

Fdi1 = find_model_function(pn1,x1,vi)
Fdi2 = find_model_function(pn2,x2,vi)

R2_1 = R2(v,Fd,pn1,x1)
R2_2 = R2(v,Fd,pn2,x2)

print('R2 - linear:    {:.5f}'.format(R2_1))
print('R2 - kvadraten: {:.5f}'.format(R2_2))

In [ ]:
fig, ax = mpl.subplots()
#fig.suptitle(r'Meritev upora pri prostem padu') # Figure title

ax.plot(v,Fd, '.g', label=r'meritev')
ax.plot(vi,Fdi1, 'r', label='linearna, $R^2$ = {:.5f}'.format(R2_1))
ax.plot(vi,Fdi2, 'b', label='kvadratna,  $R^2$ = {:.5f}'.format(R2_2))
ax.set_xlabel(r'$v$ [m/s]')
ax.set_ylabel(r'$F_d$ [N]')
ax.grid()
ax.set_aspect('equal')
ax.legend()

fig.tight_layout()
fig.savefig('01_primer-01_1.pdf')